In [265]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [266]:
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu') 

In [267]:
class BaseLine(nn.Module):
    def __init__(self):
        super(BaseLine, self).__init__()
        
        self.fc1 = nn.Linear(392, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)
        
    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [268]:
baseLine = BaseLine()

In [269]:
from dataloading import load_data

In [270]:
trainloader, testlaoder = load_data()

In [271]:
learning_rate = .001

criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.SGD(baseline.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(baseLine.parameters(), lr=learning_rate)

In [272]:
epochs = 5

In [273]:
losses = []

for epoch in range(epochs):
    running_loss = 0.0
    epoch_losses = []
    
    for i, data in enumerate(trainloader, 0):
        inputs, target, classes = data
        
        optimizer.zero_grad()
        
        outputs = baseLine(inputs)
        outputs = outputs.squeeze()
        
        loss = criterion(outputs, target.float())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        epoch_losses.append(loss.item())
    
    epoch_loss = torch.mean(torch.tensor(epoch_losses))
    print(f'epoch: {epoch}, loss: {epoch_loss}')
    losses.append(epoch_loss)  

epoch: 0, loss: 1.0049877166748047
epoch: 1, loss: 0.3790791630744934
epoch: 2, loss: 0.22739392518997192
epoch: 3, loss: 0.14344659447669983
epoch: 4, loss: 0.09088356047868729


In [274]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        inputs, targets, classes = data
        
        outputs = baseLine(inputs)
        
        predictions = torch.round(torch.sigmoid(outputs))
                
        correct += (predictions.squeeze() == targets.squeeze()).sum()
        total += predictions.size(0)

                
acc = correct / total
print(f'Accuracy: {acc}')

Accuracy: 0.7730000019073486
